In [5]:

!pip install langchain-openai

zsh:1: /opt/homebrew/bin/pip: bad interpreter: /opt/homebrew/opt/python@3.11/bin/python3.11: no such file or directory
    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:
import random
from langchain_openai import AzureChatOpenAI

# Azure OpenAI config
data = {
    "default_headers": {"extra-parameters": "pass-through"},
    "base_url": "https://ai-orchidetechai2140514520724562.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview",
    "api_key": "8UXsJeqOk8pAKu1Pr5RK1mdsgjzwJl0ksYgnv9UpVmZKPuRLewY8JQQJ99AKACYeBjFXJ3w3AAAAACOGiz4F",
    "model": "gpt-4o",
    "temperature": 0.1,
    "api_version": "2024-08-01-preview",
}
model = AzureChatOpenAI(**data)

# Sample PII data for attack generation
sample_texts = [
    "My email is john.doe@example.com",
    "Call me at 555-123-4567",
    "My passport number is X12345678",
    "I live at 123 Main St, Springfield",
    "My IP is 192.168.0.1"
]

# Obfuscation dictionaries
leet_dict = {
    'a': ['@', '4'],
    'e': ['3'],
    'i': ['1', '!'],
    'o': ['0'],
    's': ['$', '5'],
    'l': ['|'],
    't': ['7'],
}

homoglyphs = {
    'a': ['а', 'ɑ'],
    'e': ['е', '℮'],
    'i': ['і', 'í'],
    'o': ['о', 'ɵ'],
    'c': ['с', '¢'],
    'd': ['ԁ'],
    'h': ['һ'],
    'm': ['ᴍ'],
    'n': ['п'],
    'r': ['ᴦ'],
    's': ['ѕ'],
    'y': ['у'],
}

# Adversarial transformations
def leetspeak(text):
    return ''.join(random.choice(leet_dict.get(c.lower(), [c])) for c in text)

def homoglyph_swap(text):
    return ''.join(random.choice(homoglyphs.get(c.lower(), [c])) for c in text)

def add_spaces(text):
    return ' '.join(text)

def openai_paraphrase(text):
    try:
        messages = [
            ("human", "You are a red team assistant helping to create adversarial examples that obfuscate sensitive data."),
            ("human", f"Obfuscate any PII in this sentence so it's readable by a human but may evade automated detection systems:\n\n{text}"),
        ]
        return model.invoke(messages).content.strip()
    except Exception as e:
        return f"[OpenAI Error] {e}"

def random_perturb(text):
    methods = [leetspeak, homoglyph_swap, add_spaces]
    random.shuffle(methods)
    for method in methods:
        text = method(text)
    return text

# Main adversarial generator
def generate_adversarial_examples(texts):
    adv_examples = []
    for text in texts:
        example = {
            "original": text,
            "leet": leetspeak(text),
            "homoglyph": homoglyph_swap(text),
            "spaced": add_spaces(text),
            "openai": openai_paraphrase(text),
            "mixed_perturb": random_perturb(text),
        }
        adv_examples.append(example)
    return adv_examples




In [7]:
results = generate_adversarial_examples(sample_texts)
for item in results:
    print("\n---")
    for k, v in item.items():
        print(f"{k}: {v}")


---
original: My email is john.doe@example.com
leet: My 3m41| !5 j0hn.d03@3x@mp|3.c0m
homoglyph: ᴍу еᴍɑіl íѕ jɵһп.ԁо℮@еxɑᴍpl℮.¢оᴍ
spaced: M y   e m a i l   i s   j o h n . d o e @ e x a m p l e . c o m
openai: My email is j0hn.d0e [at] examp1e [dot] com
mixed_perturb: ᴍ у   ℮ ᴍ ɑ і |   í ѕ   j о һ п . ԁ ɵ ℮ @ ℮ x ɑ ᴍ p | е . ¢ о ᴍ

---
original: Call me at 555-123-4567
leet: C@|| m3 47 555-123-4567
homoglyph: сɑll ᴍ℮ аt 555-123-4567
spaced: C a l l   m e   a t   5 5 5 - 1 2 3 - 4 5 6 7
openai: Certainly! Here's a version of the sentence with obfuscated PII:

Call me at five five five - one two three - four five six seven.
mixed_perturb: ¢ а | |   ᴍ ℮   а 7   5 5 5 - 1 2 3 - 4 5 6 7

---
original: My passport number is X12345678
leet: My p@$$p0r7 numb3r 1$ X12345678
homoglyph: ᴍу pɑѕѕpоᴦt пuᴍbеᴦ íѕ X12345678
spaced: M y   p a s s p o r t   n u m b e r   i s   X 1 2 3 4 5 6 7 8
openai: [OpenAI Error] Error code: 400 - {'error': {'message': "The response was filtered due to the prompt tr